In [1]:
!pip install playwright pandas pytz
!playwright install

In [2]:
import asyncio
import pandas as pd
import datetime as dt
from pytz import timezone
from playwright.async_api import async_playwright

In [3]:
import nest_asyncio
import asyncio
nest_asyncio.apply()

In [4]:
# SF Compute price grid (keep your flow; fix "1 week" + all counts)
import re, nest_asyncio, asyncio, datetime as dt, pandas as pd
from pathlib import Path
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

nest_asyncio.apply()

GPU_TYPES = ["H100", "H200"]
DURATIONS_EXPECTED = ["1 hour", "1 day", "1 week", "1 month"]

# Strict $-anchored price AND a decimal fallback (used only when $ not in text)
PRICE_RE = re.compile(r"(?<=\$)\s*([0-9]+(?:\.[0-9]{1,3})?)")
DECIMAL_FALLBACK_RE = re.compile(r"([0-9]+(?:\.[0-9]{1,3}))")


async def _select_explore_prices_tab(page):
    try:
        await page.get_by_role("tab", name=re.compile(r"Explore Prices", re.I)).click(timeout=1500)
    except:
        try:
            await page.get_by_text(re.compile(r"Explore Prices", re.I)).first.click(timeout=1500)
        except:
            pass

async def _select_gpu(page, gpu_label: str):
    """Robustly select GPU type (H100/H200)."""
    # 1) Native <select>
    try:
        sel = page.get_by_role("combobox").first
        if await sel.count() > 0:
            await sel.select_option(label=gpu_label)
            return True
    except:
        pass
    # 2) Custom dropdown
    try:
        dd = page.get_by_role("button", name=re.compile(r"H100|H200", re.I)).first
        if await dd.count() == 0:
            dd = page.locator("div,button,[role='button']").filter(has_text=re.compile(r"H100|H200")).first
        await dd.click(timeout=1500)
        await page.get_by_text(re.compile(rf"^{gpu_label}$", re.I)).first.click(timeout=1500)
        return True
    except:
        pass
    # 3) Button toggle
    try:
        await page.get_by_role("button", name=re.compile(rf"^{gpu_label}$", re.I)).first.click(timeout=1500)
        return True
    except:
        pass
    # 4) Fallback
    try:
        await page.get_by_text(re.compile(rf"^{gpu_label}$", re.I)).first.click(timeout=1500)
        return True
    except:
        return False

async def _wait_table_update(page, expected_gpu: str):
    tbl = page.locator("table").first
    if await page.locator("table").count() == 0:
        tbl = page.locator("section, div").filter(
            has_text=re.compile("1 hour|1 day|1 week|1 month", re.I)
        ).first
    try:
        await tbl.wait_for(state="visible", timeout=6000)
    except:
        pass
    # settle after GPU switch so the row texts are fresh
    await page.wait_for_timeout(400)
    return tbl

def _slice_one_row_text(full_text: str, dur: str) -> str:
    """Take only the text segment from this duration label up to the next duration label."""
    lo = full_text.lower()
    start = lo.find(dur)
    if start == -1:
        return ""
    next_pos = len(full_text)
    for other in DURATIONS_EXPECTED:
        if other == dur:
            continue
        p = lo.find(other, start + 1)
        if p != -1:
            next_pos = min(next_pos, p)
    return full_text[start:next_pos]

# ---------- NEW: DOM fallback for rows where '$' is CSS-injected (e.g., '1 week') ----------
async def _extract_row_prices_via_dom(table_locator, dur: str, ncols: int):
    """
    Find the visual row that contains `dur` and pull texts/attributes from its cells.
    Returns a list of length ncols with floats or None.
    """
    # locate the label element first
    label_loc = table_locator.get_by_text(re.compile(rf"\b{re.escape(dur)}\b", re.I)).first
    if await label_loc.count() == 0:
        return None

    # Grab candidate cell texts/attributes from the same row container
    cell_texts = await label_loc.evaluate("""
      (el) => {
        function findRowRoot(node){
          while (node && node !== document.body) {
            if (node.tagName === 'TR') return node;
            const role = node.getAttribute && node.getAttribute('role');
            if (role && role.toLowerCase() === 'row') return node;
            const cls = (node.className || '').toString();
            if (/\\brow\\b/i.test(cls)) return node;
            node = node.parentElement;
          }
          return null;
        }
        const row = findRowRoot(el) || el.parentElement;
        if (!row) return [];

        // Prefer direct siblings after the label cell; otherwise, collect all leaf cells
        const out = [];
        const children = Array.from(row.children);
        let startIdx = children.findIndex(ch => ch.contains(el));
        if (startIdx >= 0) {
          for (let i = startIdx + 1; i < children.length; i++) {
            const c = children[i];
            const t = (c.innerText || c.textContent || '').replace(/\\s+/g,' ').trim();
            const aria = c.getAttribute && (c.getAttribute('aria-label') || '');
            const data = c.getAttribute && (c.getAttribute('data-price') || '');
            out.push([t, aria, data].filter(Boolean).join(' '));
          }
        }
        if (out.length === 0) {
          const cells = Array.from(row.querySelectorAll('td,th,a,button,div,span'));
          for (const c of cells) {
            if (c.contains(el) || el.contains(c)) continue;
            const t = (c.innerText || c.textContent || '').replace(/\\s+/g,' ').trim();
            const aria = c.getAttribute && (c.getAttribute('aria-label') || '');
            const data = c.getAttribute && (c.getAttribute('data-price') || '');
            out.push([t, aria, data].filter(Boolean).join(' '));
          }
        }
        return out;
      }
    """)

    # Parse decimals; require a decimal point to avoid '1' from '1 week'
    vals = []
    for txt in cell_texts:
        m = DECIMAL_FALLBACK_RE.search(txt)
        vals.append(float(m.group(1)) if m else None)
        if len(vals) >= ncols:
            break

    if not any(v is not None for v in vals):
        return None

    # pad to ncols
    return vals + [None] * (ncols - len(vals))

# ---------- MODIFIED: parser that prefers $ text, then DOM fallback ----------
async def _parse_grid_from_table(table_locator):
    """
    Minimal-change parser:
    - Force columns to the full set [8,16,32,64,128,256]
    - For each duration label, grab that *row* from the DOM and parse the next cells left-to-right.
    - Prefer $-anchored numbers; if missing (e.g., CSS-injected $ in 1-week links), fall back to decimals.
    """
    counts_int = [8, 16, 32, 64, 128, 256]

    rows = {}
    for dur in DURATIONS_EXPECTED:
        # Locate the row that contains exactly this duration label
        row = table_locator.locator("tr", has_text=re.compile(rf"^\s*{re.escape(dur)}\s*$", re.I)).first
        if await row.count() == 0:
            # Sometimes it's not a semantic <tr>; look for any row-like container
            row = table_locator.locator("*[role='row'], div, section").filter(
                has_text=re.compile(rf"^\s*{re.escape(dur)}\s*$", re.I)
            ).first

        # Pull texts from the next N cells after the label cell
        # We read both plain cell text and any nested link/button text/attributes.
        cell_texts = await row.evaluate("""
          (el) => {
            // Find the element that has the duration text, then read its siblings
            const textMatches = (node, dur) =>
              (node.innerText || node.textContent || '').trim().toLowerCase() === dur.toLowerCase();

            function findLabelCell(root, dur){
              const all = Array.from(root.querySelectorAll('th,td,div,span'));
              return all.find(n => textMatches(n, dur)) || root;
            }

            function collectCellStrings(nodes){
              const out = [];
              for(const n of nodes){
                const base = (n.innerText || n.textContent || '').replace(/\\s+/g,' ').trim();
                const aria = n.getAttribute?.('aria-label') || '';
                const title = n.getAttribute?.('title') || '';
                const data  = n.getAttribute?.('data-price') || '';
                // Include nested anchors/buttons in case the number lives there
                const nested = Array.from(n.querySelectorAll('a,button,span,div')).map(x =>
                  (x.innerText || x.textContent || '').replace(/\\s+/g,' ').trim()
                ).filter(Boolean).join(' ');
                out.push([base, aria, title, data, nested].filter(Boolean).join(' '));
              }
              return out;
            }

            const label = findLabelCell(el, /* dur injected below by Playwright */ '');
            // Children after label within the same row/container
            const siblings = Array.from(label.parentElement?.children || []);
            const idx = siblings.indexOf(label);
            const after = idx >= 0 ? siblings.slice(idx + 1) : [];

            // If that yields nothing (non-tabular markup), fall back to all cells under row
            const cells = after.length ? after : Array.from(el.querySelectorAll('td,th,div,span'));
            return collectCellStrings(cells);
          }
        """, arg=dur)

        # Now extract numbers from those cell strings
        vals = []
        for txt in cell_texts:
            # 1) Try $-anchored prices first
            m = PRICE_RE.search(txt)
            if m:
                vals.append(float(m.group(1)))
            else:
                # 2) Fallback to decimals (handles the 1-week link cells)
                m2 = DECIMAL_FALLBACK_RE.search(txt)
                vals.append(float(m2.group(1)) if m2 else None)
            if len(vals) >= len(counts_int):
                break

        # Pad/trim to the expected 6 columns
        vals = (vals + [None] * len(counts_int))[:len(counts_int)]
        rows[dur] = vals

    return counts_int, rows


async def scrape_sfcompute_grid(headless=True, slow_mo=0):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless, slow_mo=slow_mo)
        page = await browser.new_page()
        await page.goto("https://sfcompute.com/buy", wait_until="networkidle")
        await _select_explore_prices_tab(page)

        all_rows = []
        ts = dt.datetime.utcnow().isoformat(timespec="seconds")

        for gpu in GPU_TYPES:
            await _select_gpu(page, gpu)
            table = await _wait_table_update(page, gpu)
            counts_int, rows = await _parse_grid_from_table(table)

            for dur, vals in rows.items():
                for c, val in zip(counts_int, vals):
                    all_rows.append({
                        "ts_utc": ts,
                        "gpu_type": gpu,
                        "gpu_count": c,
                        "duration": dur,
                        "usd_per_gpu_hr": val
                    })

        await browser.close()
        return pd.DataFrame(all_rows)

# ---- Run & Save ----
df = await scrape_sfcompute_grid(headless=True)
print(df)

out_dir = Path("sfcompute_grid")
out_dir.mkdir(exist_ok=True)
today = dt.datetime.utcnow().strftime("%Y%m%d")
daily_file = out_dir / f"grid_{today}.csv"
hist_file  = out_dir / "grid_history.csv"

df.to_csv(daily_file, index=False)
if hist_file.exists():
    hist = pd.read_csv(hist_file)
    hist = pd.concat([hist, df], ignore_index=True).drop_duplicates()
    hist.to_csv(hist_file, index=False)
else:
    df.to_csv(hist_file, index=False)

print("Saved:", daily_file)
print("History file updated:", hist_file)
#CORRECT 

                 ts_utc gpu_type  gpu_count duration  usd_per_gpu_hr
0   2025-09-10T19:31:20     H100          8   1 hour            1.40
1   2025-09-10T19:31:20     H100         16   1 hour            1.41
2   2025-09-10T19:31:20     H100         32   1 hour            1.39
3   2025-09-10T19:31:20     H100         64   1 hour            1.40
4   2025-09-10T19:31:20     H100        128   1 hour            1.44
5   2025-09-10T19:31:20     H100        256   1 hour            1.55
6   2025-09-10T19:31:20     H100          8    1 day            1.40
7   2025-09-10T19:31:20     H100         16    1 day            1.40
8   2025-09-10T19:31:20     H100         32    1 day            1.40
9   2025-09-10T19:31:20     H100         64    1 day            1.40
10  2025-09-10T19:31:20     H100        128    1 day            1.40
11  2025-09-10T19:31:20     H100        256    1 day            1.77
12  2025-09-10T19:31:20     H100          8   1 week            1.40
13  2025-09-10T19:31:20     H100  

In [10]:
# === Forecast Layer (Prophet) — works locally & on GitHub, fixes timezone ===
import pandas as pd
import numpy as np
from pathlib import Path
from prophet import Prophet
import datetime as dt

# ---------- locate files (GitHub first, local fallback) ----------
hist_path = Path("docs/data/sfcompute_history.csv")
if not hist_path.exists():
    hist_path = Path("sfcompute_history.csv")  # local testing fallback

out_path = Path("docs/data/sfcompute_forecast.csv")
if not out_path.parent.exists():
    out_path = Path("sfcompute_forecast.csv")  # local fallback

print("Using history file:", hist_path.resolve())
print("Saving forecast to:", out_path.resolve())

# ---------- load & clean ----------
df = pd.read_csv(hist_path)
# expected columns: ts_utc, gpu_type, duration, gpu_count, usd_per_gpu_hr
df = df.dropna(subset=["ts_utc", "usd_per_gpu_hr"])
df["ts_utc"] = pd.to_datetime(df["ts_utc"], utc=True, errors="coerce")
df = df.dropna(subset=["ts_utc"])
df = df.sort_values("ts_utc")

# Make a tz-naive UTC column for Prophet
df["ds"] = df["ts_utc"].dt.tz_convert("UTC").dt.tz_localize(None)
df["y"]  = pd.to_numeric(df["usd_per_gpu_hr"], errors="coerce")
df = df.dropna(subset=["y"])

# remove duplicates on the same timestamp per group (keep last)
df = df.drop_duplicates(subset=["gpu_type","duration","gpu_count","ds"], keep="last")

# ---------- params ----------
HORIZON_HOURS = 24 * 7     # 7-day hourly forecast
MIN_POINTS    = 20         # skip groups with too little history
TRAIN_CLIP_D  = 30         # (optional) only last 30 days of history to speed up

results = []

# group and forecast per combination
for (gpu, dur, cnt), g in df.groupby(["gpu_type","duration","gpu_count"], dropna=False):
    g = g[["ds","y"]].sort_values("ds")

    # optional: clip to recent history
    if TRAIN_CLIP_D:
        cutoff = g["ds"].max() - pd.Timedelta(days=TRAIN_CLIP_D)
        g = g[g["ds"] >= cutoff]

    if len(g) < MIN_POINTS:
        print(f"Skipping {gpu}-{dur}-{cnt}: not enough data ({len(g)})")
        continue

    try:
        m = Prophet(daily_seasonality=True, weekly_seasonality=True)
        m.fit(g)

        future = m.make_future_dataframe(periods=HORIZON_HOURS, freq="H")
        fc = m.predict(future)[["ds","yhat","yhat_lower","yhat_upper"]].copy()

        fc["gpu_type"] = gpu
        fc["duration"] = dur
        fc["gpu_count"] = cnt
        fc["generated_utc"] = dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc).isoformat()

        results.append(fc)
        print(f"Forecast generated for {gpu}-{dur}-{cnt}")

    except Exception as e:
        # Fallback: naïve constant forecast using last observed value
        last_ds = g["ds"].max()
        last_y  = g.loc[g["ds"].idxmax(), "y"]
        future_idx = pd.date_range(last_ds + pd.Timedelta(hours=1), periods=HORIZON_HOURS, freq="H")
        fc = pd.DataFrame({
            "ds": future_idx,
            "yhat": last_y,
            "yhat_lower": last_y * 0.99,
            "yhat_upper": last_y * 1.01,
            "gpu_type": gpu,
            "duration": dur,
            "gpu_count": cnt,
            "generated_utc": dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc).isoformat(),
        })
        results.append(fc)
        print(f"⚠️ Prophet failed for {gpu}-{dur}-{cnt}: {e} — wrote naïve fallback.")

# ---------- save ----------
if results:
    forecast_all = pd.concat(results, ignore_index=True)
    # ensure ds is ISO string for the dashboard CSV
    forecast_all["ds"] = pd.to_datetime(forecast_all["ds"]).dt.strftime("%Y-%m-%d %H:%M:%S")
    forecast_all.to_csv(out_path, index=False)
    print(f"✅ Saved forecast → {out_path} ({len(forecast_all)} rows)")
else:
    print("⚠️ No forecasts were generated.")


Using history file: /Users/evieculloty/Documents/Forward Compute/sfcompute_history.csv
Saving forecast to: /Users/evieculloty/Documents/Forward Compute/docs/data/sfcompute_forecast.csv
Forecast generated for H100-1 day-8
Forecast generated for H100-1 day-16
Forecast generated for H100-1 day-32
Forecast generated for H100-1 day-64


20:39:33 - cmdstanpy - INFO - Chain [1] start processing


Forecast generated for H100-1 day-128


20:39:37 - cmdstanpy - INFO - Chain [1] done processing
20:39:37 - cmdstanpy - INFO - Chain [1] start processing
20:39:37 - cmdstanpy - INFO - Chain [1] done processing


Forecast generated for H100-1 day-256


...

Forecast generated for H200-1 week-16


20:40:19 - cmdstanpy - INFO - Chain [1] done processing
20:40:20 - cmdstanpy - INFO - Chain [1] start processing


Forecast generated for H200-1 week-32


20:40:20 - cmdstanpy - INFO - Chain [1] done processing


Forecast generated for H200-1 week-64
Forecast generated for H200-1 week-128
✅ Saved forecast → docs/data/sfcompute_forecast.csv (10938 rows)


In [12]:
# === FAST METRICS/BACKTEST (laptop-friendly) ===
import pandas as pd, numpy as np
from pathlib import Path
from prophet import Prophet
import warnings, logging

# quiet cmdstanpy/progress
logging.getLogger("cmdstanpy").disabled = True
warnings.filterwarnings("ignore")

# ----- settings (tune these for speed vs quality) -----
DO_BACKTEST     = True      # set False to skip backtest entirely
MAX_GROUPS      = 12        # cap number of groups to evaluate
ROLL_WINDOW_H   = 24        # evaluate last 24 hours only
STEP_HOURS      = 6         # fit every 6 hours (≈4 fits per group)
MIN_POINTS      = 36        # need at least 36 points to run
COUNTS_KEEP     = {8,16,32,64}
DURS_KEEP       = {"1 hour","1 day","1 week"}
TYPES_KEEP      = {"H100","H200"}

# ----- locate files -----
hist_path = Path("docs/data/sfcompute_history.csv")
if not hist_path.exists(): hist_path = Path("sfcompute_history.csv")
metrics_out = Path("docs/data/sfcompute_metrics.csv")
if not metrics_out.parent.exists(): metrics_out = Path("sfcompute_metrics.csv")

# ----- load & prep -----
df = pd.read_csv(hist_path)
df = df.dropna(subset=["ts_utc","usd_per_gpu_hr"])
df["ts_utc"] = pd.to_datetime(df["ts_utc"], utc=True)
df = df.sort_values("ts_utc")

# focus on most relevant groups to keep runtime small
df = df[df["gpu_type"].isin(TYPES_KEEP) & df["duration"].isin(DURS_KEEP) & df["gpu_count"].isin(COUNTS_KEEP)]

# Prophet-friendly columns (tz-naive)
df["ds"] = df["ts_utc"].dt.tz_convert("UTC").dt.tz_localize(None)
df["y"]  = pd.to_numeric(df["usd_per_gpu_hr"], errors="coerce")
df = df.dropna(subset=["y"])
df = df.drop_duplicates(subset=["gpu_type","duration","gpu_count","ds"], keep="last")

# choose top groups by number of observations
grp_sizes = df.groupby(["gpu_type","duration","gpu_count"]).size().sort_values(ascending=False)
keep_groups = list(grp_sizes.index[:MAX_GROUPS])

def smape(a, f):
    a, f = np.array(a), np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    denom[denom==0] = 1e-9
    return float(np.mean(np.abs(f - a) / denom) * 100)

rows = []

for (gpu,dur,cnt) in keep_groups:
    g = df[(df.gpu_type==gpu)&(df.duration==dur)&(df.gpu_count==cnt)][["ds","y"]].sort_values("ds").reset_index(drop=True)
    n = len(g)
    if n < MIN_POINTS:
        rows.append(dict(gpu_type=gpu,duration=dur,gpu_count=cnt,n=n,
                         mae=np.nan,mape=np.nan,smape=np.nan,
                         mae_naive=np.nan,skill=np.nan,pi_coverage=np.nan,
                         ensemble_w=0.5))
        continue

    if not DO_BACKTEST:
        # quick metrics using one fit + in-sample prediction (very fast, approximate)
        m = Prophet(daily_seasonality=True, weekly_seasonality=True,
                    growth='flat', n_changepoints=5, changepoint_prior_scale=0.05)
        m.fit(g.rename(columns={"ds":"ds","y":"y"}))
        fc = m.predict(g[["ds"]])
        yhat = fc["yhat"].to_numpy()
        y    = g["y"].to_numpy()
        naive = np.r_[np.nan, y[:-1]]
        mask = ~np.isnan(naive)
        mae = np.mean(np.abs(yhat[mask]-y[mask]))
        mae_nv = np.mean(np.abs(naive[mask]-y[mask]))
        skill = 1 - mae/mae_nv if mae_nv>0 else np.nan
        rows.append(dict(gpu_type=gpu,duration=dur,gpu_count=cnt,n=n,
                         mae=mae, mape=float(np.mean(np.abs((yhat[mask]-y[mask]) / np.clip(y[mask],1e-9,None)))*100),
                         smape=smape(y[mask], yhat[mask]), mae_naive=mae_nv,
                         skill=skill, pi_coverage=np.nan, ensemble_w=max(0.0, min(1.0, 0.5 + 0.5*(skill if np.isfinite(skill) else 0)))))
        continue

    # true (but small) walk-forward over last ROLL_WINDOW_H hours, step STEP_HOURS
    end_time   = g["ds"].max()
    start_time = end_time - pd.Timedelta(hours=ROLL_WINDOW_H)
    g_recent = g[g["ds"] >= start_time].copy()
    if len(g_recent) < 8:   # too little in the recent slice
        g_recent = g.tail(24).copy()

    preds, lowers, uppers, actuals, naive_preds = [], [], [], [], []
    # roll in steps
    idxs = np.arange(g.index.get_indexer(g_recent.index)[0], len(g), STEP_HOURS)
    idxs = idxs[idxs>0]  # need at least 1 train obs

    for t in idxs:
        train = g.iloc[:t].copy()
        test  = g.iloc[t:t+1].copy()

        m = Prophet(daily_seasonality=True, weekly_seasonality=True,
                    growth='flat', n_changepoints=5, changepoint_prior_scale=0.05)
        m.fit(train.rename(columns={"ds":"ds","y":"y"}))
        fc = m.predict(test[["ds"]])

        preds.append(float(fc["yhat"].iloc[0]))
        lowers.append(float(fc["yhat_lower"].iloc[0]))
        uppers.append(float(fc["yhat_upper"].iloc[0]))
        actuals.append(float(test["y"].iloc[0]))
        naive_preds.append(float(train["y"].iloc[-1]))

    actuals = np.array(actuals); preds = np.array(preds); naive = np.array(naive_preds)
    mae      = float(np.mean(np.abs(preds - actuals)))
    mape     = float(np.mean(np.abs((preds - actuals) / np.clip(actuals, 1e-9, None))) * 100)
    smape_v  = smape(actuals, preds)
    mae_nv   = float(np.mean(np.abs(naive - actuals)))
    skill    = float(1.0 - (mae / mae_nv)) if mae_nv > 0 else np.nan
    cover    = float(np.mean((actuals >= np.array(lowers)) & (actuals <= np.array(uppers)))) if len(actuals) else np.nan
    ensemble_w = max(0.0, min(1.0, 0.5 + 0.5*(skill if np.isfinite(skill) else 0)))

    rows.append(dict(gpu_type=gpu,duration=dur,gpu_count=cnt,n=n,
                     mae=mae, mape=mape, smape=smape_v,
                     mae_naive=mae_nv, skill=skill, pi_coverage=cover,
                     ensemble_w=ensemble_w))

metrics = pd.DataFrame(rows)
metrics.to_csv(metrics_out, index=False)
print(f"✅ FAST metrics saved → {metrics_out} ({len(metrics)} groups, DO_BACKTEST={DO_BACKTEST})")


✅ FAST metrics saved → docs/data/sfcompute_metrics.csv (12 groups, DO_BACKTEST=True)


In [21]:
# === BLEND (final, bulletproof) ===
from pathlib import Path
import pandas as pd, numpy as np

KEYS = ["gpu_type","duration","gpu_count"]

# --- paths ---
fore_path = Path("docs/data/sfcompute_forecast.csv")
if not fore_path.exists(): fore_path = Path("sfcompute_forecast.csv")

hist_path = Path("docs/data/sfcompute_history.csv")
if not hist_path.exists(): hist_path = Path("sfcompute_history.csv")

met_path = Path("docs/data/sfcompute_metrics.csv")
if not met_path.exists(): met_path = Path("sfcompute_metrics.csv")

# --- load forecast & history ---
fa = pd.read_csv(fore_path)
df = pd.read_csv(hist_path)

# normalise types on keys
for k in ("gpu_type","duration"):
    fa[k] = fa[k].astype(str)
    df[k] = df[k].astype(str)
fa["gpu_count"] = pd.to_numeric(fa["gpu_count"], errors="coerce").astype("Int64")
df["gpu_count"] = pd.to_numeric(df["gpu_count"], errors="coerce").astype("Int64")

# prep history
df["ts_utc"] = pd.to_datetime(df["ts_utc"], utc=True, errors="coerce")
df = df.dropna(subset=["ts_utc","usd_per_gpu_hr"]).sort_values("ts_utc")
df["ds"] = df["ts_utc"].dt.tz_convert("UTC").dt.tz_localize(None)
df["y"]  = pd.to_numeric(df["usd_per_gpu_hr"], errors="coerce")
df = df.dropna(subset=["y"])

# --- weights from metrics (optional) ---
if met_path.exists():
    met = pd.read_csv(met_path)
    for k in ("gpu_type","duration"):
        if k in met: met[k] = met[k].astype(str)
    if "gpu_count" in met:
        met["gpu_count"] = pd.to_numeric(met["gpu_count"], errors="coerce").astype("Int64")
    if "ensemble_w" not in met.columns:
        skill = pd.to_numeric(met.get("skill", 0.0), errors="coerce").fillna(0.0)
        met["ensemble_w"] = np.clip(0.5 + 0.5*skill, 0.0, 1.0)
    w_df = met[KEYS + ["ensemble_w"]].dropna(subset=["gpu_count"])
else:
    w_df = fa[KEYS].drop_duplicates().assign(ensemble_w=0.5)

fa = fa.merge(w_df, on=KEYS, how="left")
if "ensemble_w" not in fa.columns:
    fa["ensemble_w"] = 0.5
fa["ensemble_w"] = pd.to_numeric(fa["ensemble_w"], errors="coerce").fillna(0.5)

# --- last observed value per group (naïve) ---
last_y = (df.sort_values("ds")
            .groupby(KEYS)["y"].last().rename("y_last").reset_index())
fa = fa.merge(last_y, on=KEYS, how="left")

# --- recent bounds per group (last 48 samples) ---
g = df.sort_values("ds").groupby(KEYS)["y"]
bmin = g.apply(lambda s: float(s.tail(48).min()) if len(s) else np.nan).rename("recent_min").reset_index()
bmax = g.apply(lambda s: float(s.tail(48).max()) if len(s) else np.nan).rename("recent_max").reset_index()
bounds = bmin.merge(bmax, on=KEYS, how="outer")
fa = fa.merge(bounds, on=KEYS, how="left")

# --- ensure columns exist before fill (prevents KeyErrors) ---
for col in ["y_last","recent_min","recent_max"]:
    if col not in fa.columns:
        fa[col] = np.nan

# --- fill fallbacks ---
fa["y_last"] = pd.to_numeric(fa["y_last"], errors="coerce")
fa["y_last"].fillna(pd.to_numeric(fa["yhat"], errors="coerce"), inplace=True)

fa["recent_min"] = pd.to_numeric(fa["recent_min"], errors="coerce")
fa["recent_max"] = pd.to_numeric(fa["recent_max"], errors="coerce")
fa["recent_min"].fillna(fa["y_last"]*0.99, inplace=True)
fa["recent_max"].fillna(fa["y_last"]*1.01, inplace=True)

# --- blend + clip ---
yhat_num = pd.to_numeric(fa["yhat"], errors="coerce")
fa["yhat"] = (fa["ensemble_w"]*yhat_num + (1-fa["ensemble_w"])*fa["y_last"]).clip(
    lower=fa["recent_min"]*0.98, upper=fa["recent_max"]*1.02
)

# --- save (overwrite) ---
fa.drop(columns=["y_last","recent_min","recent_max"], inplace=True, errors="ignore")
fa.to_csv(fore_path, index=False)
print(f"✅ Blended forecast saved → {fore_path} ({len(fa)} rows)")


✅ Blended forecast saved → docs/data/sfcompute_forecast.csv (10938 rows)


In [22]:
# === ROI / Cost Catalog (from latest prices) ===
import pandas as pd, numpy as np
from pathlib import Path

DATA_DIRS = [Path("docs/data"), Path(".")]
def first_existing(*names):
    for d in DATA_DIRS:
        for n in names:
            p = d / n
            if p.exists(): return p
    return None

hist_path = first_existing("sfcompute_history.csv", "sfcompute_history.parquet")
latest_path = first_existing("sfcompute_latest.csv")
out_path = (Path("docs/data") if (Path("docs")).exists() else Path(".")) / "roi_catalog.csv"

# Load latest; fallback to last row per group from history
if latest_path:
    latest = pd.read_csv(latest_path)
    # expected: ts_utc, gpu_type, duration, gpu_count, usd_per_gpu_hr
else:
    df = pd.read_csv(hist_path)
    df["ts_utc"] = pd.to_datetime(df["ts_utc"], utc=True)
    latest = (df.sort_values("ts_utc")
                .groupby(["gpu_type","duration","gpu_count"], as_index=False)
                .tail(1)
                [["gpu_type","duration","gpu_count","usd_per_gpu_hr","ts_utc"]]
                .rename(columns={"ts_utc":"asof_utc"}))

# Normalise and compute hours per duration
roi = latest.copy()
roi["gpu_count"] = pd.to_numeric(roi["gpu_count"], errors="coerce").astype("Int64")
roi["usd_per_gpu_hr"] = pd.to_numeric(roi["usd_per_gpu_hr"], errors="coerce")

HOURS_PER_MONTH = 730  # standard billing month
HOURS_BY_DURATION = {
    "1 hour": 1,
    "1 day": 24,
    "1 week": 7*24,
    "1 month": HOURS_PER_MONTH
}

roi["hours_per_unit"] = roi["duration"].map(HOURS_BY_DURATION).fillna(1)
roi["monthly_hours"]  = HOURS_PER_MONTH
# monthly cost for a given fleet size N is: price/hr * monthly_hours * gpu_count_multiplier
roi["monthly_cost_usd_at_100pct_util"] = roi["usd_per_gpu_hr"] * roi["monthly_hours"] * roi["gpu_count"]

# add a few standard utilisation presets the UI can pick from (50/75/100%)
out_rows = []
for u in (0.5, 0.75, 1.0):
    tmp = roi.copy()
    tmp["utilisation"] = u
    tmp["monthly_cost_usd"] = tmp["monthly_cost_usd_at_100pct_util"] * u
    out_rows.append(tmp)

roi_out = pd.concat(out_rows, ignore_index=True)
roi_out = roi_out[["gpu_type","duration","gpu_count","usd_per_gpu_hr","utilisation","monthly_cost_usd","monthly_hours"]]
roi_out.to_csv(out_path, index=False)
print(f"✅ ROI catalog written → {out_path} ({len(roi_out)} rows)")


✅ ROI catalog written → docs/data/roi_catalog.csv (144 rows)


In [23]:
# === Budget Simulation tables (latest price table + example scenarios) ===
import pandas as pd
from pathlib import Path

DATA_DIRS = [Path("docs/data"), Path(".")]
def first_existing(*names):
    for d in DATA_DIRS:
        for n in names:
            p = d / n
            if p.exists(): return p
    return None

latest_path = first_existing("sfcompute_latest.csv")
hist_path   = first_existing("sfcompute_history.csv")
out_prices  = (Path("docs/data") if (Path("docs")).exists() else Path(".")) / "price_table_latest.csv"
out_sims    = (Path("docs/data") if (Path("docs")).exists() else Path(".")) / "budget_scenarios.csv"

if latest_path:
    price_tbl = pd.read_csv(latest_path)
else:
    df = pd.read_csv(hist_path)
    df["ts_utc"] = pd.to_datetime(df["ts_utc"], utc=True)
    price_tbl = (df.sort_values("ts_utc")
                   .groupby(["gpu_type","duration","gpu_count"], as_index=False)
                   .tail(1)
                   [["gpu_type","duration","gpu_count","usd_per_gpu_hr","ts_utc"]]
                   .rename(columns={"ts_utc":"asof_utc"}))

price_tbl["gpu_count"] = pd.to_numeric(price_tbl["gpu_count"], errors="coerce").astype("Int64")
price_tbl["usd_per_gpu_hr"] = pd.to_numeric(price_tbl["usd_per_gpu_hr"], errors="coerce")
price_tbl.to_csv(out_prices, index=False)
print(f"✅ Latest price table → {out_prices} ({len(price_tbl)} rows)")

# Example budget scenarios the UI can show (edit freely)
scenarios = pd.DataFrame([
    {"label":"H100 • 8 GPUs • 1h (spot) • 100% util",  "gpu_type":"H100","gpu_count":8,"duration":"1 hour","util":1.0},
    {"label":"H100 • 128 GPUs • 1w (reserved) • 80%",  "gpu_type":"H100","gpu_count":128,"duration":"1 week","util":0.8},
    {"label":"H200 • 32 GPUs • 1d (reserved) • 70%",   "gpu_type":"H200","gpu_count":32,"duration":"1 day","util":0.7},
])

HOURS_PER_MONTH = 730
sim = scenarios.merge(price_tbl, on=["gpu_type","duration","gpu_count"], how="left")
sim["usd_per_gpu_hr"] = pd.to_numeric(sim["usd_per_gpu_hr"], errors="coerce")
sim["monthly_cost_usd"] = sim["usd_per_gpu_hr"] * HOURS_PER_MONTH * sim["gpu_count"] * sim["util"]
sim = sim[["label","gpu_type","duration","gpu_count","util","usd_per_gpu_hr","monthly_cost_usd"]]
sim.to_csv(out_sims, index=False)
print(f"✅ Budget scenarios → {out_sims} ({len(sim)} rows)")


✅ Latest price table → docs/data/price_table_latest.csv (48 rows)
✅ Budget scenarios → docs/data/budget_scenarios.csv (3 rows)


In [24]:
# === Price Alerts (uses competitor median if available; else 7d SF rolling median) ===
import pandas as pd, numpy as np
from pathlib import Path

DATA_DIRS = [Path("docs/data"), Path(".")]
def first_existing(*names):
    for d in DATA_DIRS:
        for n in names:
            p = d / n
            if p.exists(): return p
    return None

latest_path   = first_existing("sfcompute_latest.csv")
hist_path     = first_existing("sfcompute_history.csv")
bench_path    = first_existing("competitor_median.csv")  # optional
out_alerts    = (Path("docs/data") if (Path("docs")).exists() else Path(".")) / "alerts.csv"
THRESH = 0.05  # 5% below benchmark

# load latest
if latest_path:
    latest = pd.read_csv(latest_path)
else:
    df = pd.read_csv(hist_path)
    df["ts_utc"] = pd.to_datetime(df["ts_utc"], utc=True)
    latest = (df.sort_values("ts_utc")
                .groupby(["gpu_type","duration","gpu_count"], as_index=False)
                .tail(1)
                [["gpu_type","duration","gpu_count","usd_per_gpu_hr","ts_utc"]]
                .rename(columns={"ts_utc":"asof_utc"}))

latest["gpu_count"] = pd.to_numeric(latest["gpu_count"], errors="coerce").astype("Int64")
latest["usd_per_gpu_hr"] = pd.to_numeric(latest["usd_per_gpu_hr"], errors="coerce")

# build benchmark
if bench_path:
    bench = pd.read_csv(bench_path)
    for k in ("gpu_type","duration"): bench[k] = bench[k].astype(str)
    bench["gpu_count"] = pd.to_numeric(bench["gpu_count"], errors="coerce").astype("Int64")
    bench = bench.rename(columns={"median_price_hourly":"benchmark_price"})
else:
    # internal benchmark = 7d rolling median of SF prices per group
    df = pd.read_csv(hist_path)
    df["ts_utc"] = pd.to_datetime(df["ts_utc"], utc=True)
    df = df.sort_values("ts_utc")
    df["usd_per_gpu_hr"] = pd.to_numeric(df["usd_per_gpu_hr"], errors="coerce")

    # approx 7d window in hours; if irregular, use last N=168 points
    def last7d_median(g):
        s = g["usd_per_gpu_hr"].tail(168)
        return pd.Series({"benchmark_price": s.median()})

    bench = (df.groupby(["gpu_type","duration","gpu_count"])
               .apply(last7d_median)
               .reset_index())
    for k in ("gpu_type","duration"): bench[k] = bench[k].astype(str)
    bench["gpu_count"] = pd.to_numeric(bench["gpu_count"], errors="coerce").astype("Int64")

alerts = latest.merge(bench, on=["gpu_type","duration","gpu_count"], how="left")
alerts["delta_pct"] = (alerts["usd_per_gpu_hr"] - alerts["benchmark_price"]) / alerts["benchmark_price"]
alerts["is_below_benchmark"] = alerts["delta_pct"] <= -THRESH

alerts_out = alerts[alerts["is_below_benchmark"]==True].copy()
alerts_out = alerts_out[["gpu_type","duration","gpu_count","usd_per_gpu_hr","benchmark_price","delta_pct"]]
alerts_out.to_csv(out_alerts, index=False)
print(f"✅ Alerts written → {out_alerts} ({len(alerts_out)} rows)")


✅ Alerts written → docs/data/alerts.csv (0 rows)
